# APRENDENDO PANDAS

* Import Fase

In [109]:

import pandas as pd
import requests
from bs4 import BeautifulSoup



* Request Fase

In [110]:
# Request web page
resp = requests.get("https://www.fundsexplorer.com.br/ranking")

# Get html content
html = resp.text


* Parser Functions

In [111]:


def tableDataText(table):
    """Parses a html segment started with tag <table> followed
    by multiple <tr> (table rows) and inner <td> (table data) tags.
    It returns a list of rows with inner columns.
    Accepts only one <th> (table header/data) in the first row.
    """

    def rowgetDataText(tr, coltag="td"):  # td (data) or th (header)
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]

    rows = []
    trs = table.find_all("tr")
    headerow = rowgetDataText(trs[0], "th")
    if headerow:  # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs:  # for every table row
        rows.append(rowgetDataText(tr, "td"))  # data row
    return rows


def parser_text(text: str):
    """ Parser string text e.g R$ 99,01 to 99.01"""
    if text == "N/A":
        return 0

    text = str(text).replace(" ", "")
    text = str(text).replace("(", "")
    text = str(text).replace(")", "")
    text = str(text).replace("R$", "")
    text = str(text).replace("%", "")
    text = str(text).replace(".", "")
    text = str(text).replace(",", ".")
    text = str(text).replace("/", "")

    return text



* Soap Fase

In [112]:

# Create soap instance
soup = BeautifulSoup(html, features="html.parser")

# Find table in html content
htmltable = soup.find("table", {"id": "table-ranking"})
table = tableDataText(htmltable)


* Carregando DataFrame

In [113]:
# Parseamentos dos titulos das colunas
colunas = map(parser_text, table[0])

# Iniciando dataframe
df = pd.DataFrame(table[1:], columns=colunas)



# Operações com dataframe

In [114]:

# Parsear varias colunas via callback com apply
for coluna in df.columns:
    df.loc[:, coluna] = df[coluna].apply(parser_text)


In [115]:

# Buscar com condicao em coluna expecifica
df.loc[df["Códigodofundo"] == "MXRF11"]


,Códigodofundo,Setor,PreçoAtual,LiquidezDiária,Dividendo,DividendYield,DY3MAcumulado,DY6MAcumulado,DY12MAcumulado,DY3MMédia,...,PatrimônioLíq,VPA,PVPA,DYPatrimonial,VariaçãoPatrimonial,RentabPatrnoPeríodo,RentabPatrAcumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
74,MXRF11,Híbrido,9.65,4455770,0.11,1.11,3.31,6.17,10.84,1.10,...,2302693543.19,10.19,0.95,0.98,-0.02,0.96,2.22,0,0,0


In [116]:

# Buscar com condicao com colunas expecificas
# Exibindo resposta com colunas expessificas passadas via lista
df.loc[df["Códigodofundo"] == "MXRF11", ["Códigodofundo", "PreçoAtual"]]


,Códigodofundo,PreçoAtual
74,MXRF11,9.65


In [117]:

# Converter tipo de dados da coluna
df.loc[:, "DY12MMédia"] = df["DY12MMédia"].astype(float)
df.loc[:, "PVPA"] = df["PVPA"].astype(float)
df.loc[:, "QuantidadeAtivos"] = df["QuantidadeAtivos"].astype(float)
df.loc[:, "VacânciaFísica"] = df["VacânciaFísica"].astype(float)


In [118]:

# Query
colunas = [
    "Códigodofundo",
    "Setor",
    "PreçoAtual",
    "DividendYield",
    "DY12MMédia",
    "RentabAcumulada",
    "PatrimônioLíq",
    "PVPA",
    "VacânciaFísica",
    "VacânciaFinanceira",
    "QuantidadeAtivos",
]
pesquisa = df.query("DY12MMédia > 0.5 & QuantidadeAtivos > 10 & VacânciaFísica < 10")
pesquisa = pesquisa.loc[:, colunas].sort_values(
    ["DY12MMédia", "DividendYield"], ascending=False
)
pesquisa


,Códigodofundo,Setor,PreçoAtual,DividendYield,DY12MMédia,RentabAcumulada,PatrimônioLíq,PVPA,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
111,BNFS11,Outros,118.30,1.19,1.00,-0.28,69417286.55,1.19,0.00,0,18.0
102,BBPO11,LajesCorporativas,80.09,1.06,0.95,-6.17,1512931778.94,0.84,0.00,0.00,64.0
170,RBVA11,Outros,89.99,0.99,0.91,-1.48,1193288764.70,0.87,0.70,0,72.0
95,BBRC11,Outros,98.90,0.99,0.88,2.60,170397413.70,0.92,5.37,1.08,20.0
131,TRXF11,Outros,98.70,0.86,0.77,1.44,708147194.50,1.01,0.00,0,44.0
185,QAGR11,Outros,46.81,0.84,0.77,-1.48,300831716.41,0.78,0.00,0,12.0
189,GGRC11,Logística,107.85,0.85,0.75,-2.35,1012027467.97,0.83,0.00,0.00,15.0
177,HGLG11,Logística,162.43,0.67,0.75,0.30,3501807619.41,1.09,6.90,6.50,17.0
266,BTLG11,Logística,100.50,0.71,0.69,2.94,1504876179.16,1.02,0.00,2.00,16.0
179,HGRU11,Híbrido,117.28,0.69,0.67,6.77,2191214311.59,0.99,0.00,0.00,17.0


In [119]:

# Retorna lista de valores unicos em uma coluna
df.loc[:, "Setor"].unique()


array(['Shoppings', 'TítuloseValMob', 'LajesCorporativas', 'Híbrido',
       'Hospital', 'Logística', '', 'Hotel', 'Residencial', 'Outros'],
      dtype=object)

In [144]:

# Metodo de iterecao
for row in pesquisa.itertuples():
    print('',end='\n')
    for index, column in enumerate(pesquisa.columns):
        print(column,row[index],end=', ')
    break

Index(['Códigodofundo', 'Setor', 'PreçoAtual', 'DividendYield', 'DY12MMédia',
       'RentabAcumulada', 'PatrimônioLíq', 'PVPA', 'VacânciaFísica',
       'VacânciaFinanceira', 'QuantidadeAtivos'],
      dtype='object')

Códigodofundo 111, Setor BNFS11, PreçoAtual Outros, DividendYield 118.30, DY12MMédia 1.19, RentabAcumulada 1.0, PatrimônioLíq -0.28, PVPA 69417286.55, VacânciaFísica 1.19, VacânciaFinanceira 0.0, QuantidadeAtivos 0, 
Códigodofundo 102, Setor BBPO11, PreçoAtual LajesCorporativas, DividendYield 80.09, DY12MMédia 1.06, RentabAcumulada 0.95, PatrimônioLíq -6.17, PVPA 1512931778.94, VacânciaFísica 0.84, VacânciaFinanceira 0.0, QuantidadeAtivos 0.00, 
Códigodofundo 170, Setor RBVA11, PreçoAtual Outros, DividendYield 89.99, DY12MMédia 0.99, RentabAcumulada 0.91, PatrimônioLíq -1.48, PVPA 1193288764.70, VacânciaFísica 0.87, VacânciaFinanceira 0.7, QuantidadeAtivos 0, 
Códigodofundo 95, Setor BBRC11, PreçoAtual Outros, DividendYield 98.90, DY12MMédia 0.99, RentabAcumulada 0.88, 

In [121]:


# Gravar no excel
pesquisa.to_excel("output.xlsx")
